In [16]:
import json
from collections import Counter

import numpy as np
import tqdm
import pandas as pd
import torch
from torchtext.data.utils import get_tokenizer, ngrams_iterator
from torchtext.datasets import TextClassificationDataset
from torchtext.vocab import Vocab

## read data

In [2]:
df = pd.read_csv('data/Reviews.csv')
df = df[['Score', 'Text']]
df = df.dropna()
df = df.drop_duplicates('Text')

In [3]:
df.head()

,Score,Text
0,5,I have bought several of the Vitality canned d...
1,1,Product arrived labeled as Jumbo Salted Peanut...
2,4,This is a confection that has been around a fe...
3,2,If you are looking for the secret ingredient i...
4,5,Great taffy at a great price. There was a wid...


In [4]:
df['Text'].describe()

count                                                393579
unique                                               393579
top       Recently purchased several cans of this for in...
freq                                                      1
Name: Text, dtype: object

In [5]:
df['Score'].unique()

array([5, 1, 4, 2, 3])

## build dataset

In [6]:
labels, reviews = df['Score'].to_numpy(), df['Text'].to_numpy()

make splits

In [7]:
def split_data(arrays, ratios=(0.7, 0.2, 0.1)):
    data_len = arrays[0].shape[0]
    assert all(a.shape[0] == data_len for a in arrays[1:])
    sizes = [r / sum(ratios) for r in ratios]
    sizes = [int(s * data_len) for s in sizes[:-1]]
    sizes.append(data_len - sum(sizes))
    start = 0
    finish = 0
    splits = []
    for s in sizes:
        finish += s
        splits.append([a[start:finish] for a in arrays])
        start += s
    return splits

train, valid, test = split_data((labels, reviews), (0.7, 0.2, 0.1))

In [12]:
def data_merge(data):
    labels, text = data
    return [{'label': int(l), 'text': row} for l, row in zip(labels, text)]

train = data_merge(train)
valid = data_merge(valid)
test = data_merge(test)

ValueError: too many values to unpack (expected 2)

In [20]:
del df

tokenize text in datasets, add bigrams

In [ ]:
! python -m spacy download en

In [13]:
def data_tokenize(data, tokenizer, lower, ngrams):
    tokenizer = get_tokenizer(tokenizer)
    for entry in tqdm.tqdm(data, 'lines', len(data)):
        if lower:
            entry['text'] = entry['text'].lower()
        entry['text'] = tokenizer(entry['text'])
        entry['text'] = list(ngrams_iterator(entry['text'], ngrams))
    return data

tokenizer = 'spacy'
lower = True
ngrams = 2

train = data_tokenize(train, tokenizer, lower, ngrams)

lines: 100%|██████████| 275505/275505 [01:21<00:00, 3400.75it/s]


In [15]:
valid = data_tokenize(valid, tokenizer, lower, ngrams)
test = data_tokenize(test, tokenizer, lower, ngrams)

lines: 100%|██████████| 39359/39359 [00:11<00:00, 3367.05it/s]


save just in case

In [18]:
with open('train_tokenized.json', 'wt') as file:
    json.dump(train, file)

with open('valid_tokenized.json', 'wt') as file:
    json.dump(valid, file)

with open('test_tokenized.json', 'wt') as file:
    json.dump(test, file)

create vocabulary

In [19]:
def build_vocab(data, max_size=10000):
    counter = Counter()
    for label, row in data:
        counter.update(row)
    return Vocab(counter, max_size)

vocab = build_vocab(train)

create torch datasets

ValueError: not enough values to unpack (expected 2, got 0)